In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
data_dir = '../raw_data'

In [5]:
ds = tf.keras.utils.image_dataset_from_directory(
  data_dir)

Found 55448 files belonging to 39 classes.


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for images, labels in ds.take(2):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(ds.class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
classes = ds.class_names

In [26]:
classes = ds.class_names
numbers= np.arange(1,40,1)
dict_leaves = {classes[i]: numbers[i] for i in range(len(classes))}
dict_leaves

{'Apple___Apple_scab': 1,
 'Apple___Black_rot': 2,
 'Apple___Cedar_apple_rust': 3,
 'Apple___healthy': 4,
 'Background_without_leaves': 5,
 'Blueberry___healthy': 6,
 'Cherry___Powdery_mildew': 7,
 'Cherry___healthy': 8,
 'Corn___Cercospora_leaf_spot Gray_leaf_spot': 9,
 'Corn___Common_rust': 10,
 'Corn___Northern_Leaf_Blight': 11,
 'Corn___healthy': 12,
 'Grape___Black_rot': 13,
 'Grape___Esca_(Black_Measles)': 14,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 15,
 'Grape___healthy': 16,
 'Orange___Haunglongbing_(Citrus_greening)': 17,
 'Peach___Bacterial_spot': 18,
 'Peach___healthy': 19,
 'Pepper,_bell___Bacterial_spot': 20,
 'Pepper,_bell___healthy': 21,
 'Potato___Early_blight': 22,
 'Potato___Late_blight': 23,
 'Potato___healthy': 24,
 'Raspberry___healthy': 25,
 'Soybean___healthy': 26,
 'Squash___Powdery_mildew': 27,
 'Strawberry___Leaf_scorch': 28,
 'Strawberry___healthy': 29,
 'Tomato___Bacterial_spot': 30,
 'Tomato___Early_blight': 31,
 'Tomato___Late_blight': 32,
 'Tomato_

In [ ]:
ds.element_spec

In [ ]:
# y= ds.class_names
# X= 

In [ ]:
sample = next(iter(ds))
sample

In [ ]:
#tf.keras.layers.Rescaling(1./255)


In [33]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    #label_mode=None,
    #class_names="class_names",
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="both")  

Found 55448 files belonging to 39 classes.
Using 44359 files for training.
Using 11089 files for validation.


In [ ]:
# train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
#     'temp',
#     labels='inferred',
#     #label_mode=None,
#     #class_names="class_names",
#     color_mode='rgb',
#     batch_size=32,
#     image_size=(256, 256),
#     shuffle=True,
#     seed=123,
#     validation_split=0.1,
#     subset="both") 

In [34]:
len(ds.class_names)

39

In [ ]:
# val_ds = tf.keras.utils.image_dataset_from_directory(
#     data_dir,
#     labels='inferred',
#     color_mode='rgb',
#     batch_size=32,
#     image_size=(256, 256),
#     shuffle=False,
#     seed=123,
#     validation_split=0.2,
#     subset="validation")

In [35]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
# ds.as_numpy_iterator().next()

In [ ]:
# Split the dataset into training, test, and validation sets
# temp = ds
# test_dataset = tf.keras.utils.image_dataset_from_directory(
#     data_dir,
#     image_size=(256, 256),
#     batch_size=32,
#     validation_split=0.2,
#     subset="validation",
#     seed=42
# )

/

In [ ]:
# train_to_numpy = list(train_dataset.as_numpy_iterator())
# shape = tf.shape(dataset_to_numpy)
# print(shape)

In [36]:
from tensorflow.keras import models, losses
from tensorflow.keras import Sequential, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Rescaling, RandomFlip, RandomRotation

model = models.Sequential()

model.add(layers.Rescaling(1./255))
model.add(layers.RandomFlip("horizontal_and_vertical"))
model.add(layers.RandomRotation(0.2))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Conv2D(32, kernel_size=(3), activation='relu')) # kernel_size = 3 <==> (3, 3)
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(39, activation='softmax')) 

#model.summary() 

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=2,  # Use early stopping in practice
          batch_size=32, 
          verbose=1)

In [ ]:
model.summary()

In [37]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


# Loading the weights of a pre-trained ConvNet
pretrained = vgg16.VGG16(weights='imagenet', 
                         include_top=False,
                         input_shape=(256, 256, 3))

pretrained.trainable = False
# Optional, if you have time and want to re-train these weights, you can set it to True.

# Flatten + Intermediate Dense Layer + Predictive Layer
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(100, activation='relu')
prediction_layer = layers.Dense(38, activation='softmax')

model1 = Sequential([
    pretrained, 
    tf.keras.layers.Rescaling(1./255, input_shape=(256,256,3)),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.CenterCrop(height=224, width=224), 
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.Conv2D(64, 4, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(246, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(38, activation='softmax')
])

In [38]:
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 rescaling_2 (Rescaling)     (None, 8, 8, 512)         0         
                                                                 
 random_flip_2 (RandomFlip)  (None, 8, 8, 512)         0         
                                                                 
 center_crop_1 (CenterCrop)  (None, 224, 224, 512)     0         
                                                                 
 random_rotation_2 (RandomRo  (None, 224, 224, 512)    0         
 tation)                                                         
                                                                 
 conv2d_4 (Conv2D)           (None, 221, 221, 64)      524352    
                                                      

In [39]:
model1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer='adam',
              metrics=['accuracy'])

model1.fit(train_ds,
          validation_data=val_ds,
          epochs=1,  # Use early stopping in practice
          batch_size=32, 
          verbose=1)

/home/alice/.pyenv/versions/3.10.6/envs/LeafScan/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_3/center_crop_1/cond/gradients/grad_ys_0_indices:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_3/center_crop_1/cond/gradients/grad_ys_0_values:0", shape=(None, 224, 224, 512), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_3/center_crop_1/cond/gradients/grad_ys_0_shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


2023-06-07 13:50:10.095986: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (mklcpu) ran out of memory trying to allocate 3.06GiB (rounded to 3288334336)requested by op sequential_3/random_rotation_2/loop_body/GatherV2/pfor/GatherV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-06-07 13:50:10.106794: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for mklcpu
2023-06-07 13:50:10.106827: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-06-07 13:50:10.106831: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-06-07 13

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_3/random_rotation_2/loop_body/GatherV2/pfor/GatherV2' defined at (most recent call last):
    File "/home/alice/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/alice/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/alice/.pyenv/versions/3.10.6/envs/LeafScan/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/alice/.pyenv/versions/3.10.6/envs/LeafScan/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/alice/.pyenv/versions/3.10.6/envs/LeafScan/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/alice/.pyenv/versions/3.10.6/envs/LeafScan/lib/